In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import pandas as pd
import numpy as np
import pickle

from model_classes import SkipGramModel, Node2VecModel, SFCModel, GAEModel, GATEncoder, GCNEncoder
from pipelines.utils import ROOT_DIR, load_config
from models.utils import meters2lonlat, lonlat2meters


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
data_config = load_config(name='porto', ctype="dataset")

# Load CellSpace
dataset_cell_file = f"{data_config['city']}_cell{int(data_config['cell_size'])}_cellspace.pkl"
file_path = os.path.join(ROOT_DIR, "models/road_embs/", dataset_cell_file)
with open(file_path, 'rb') as fh:
    cs = pickle.load(fh)

# Load Feats Matrix
feats_mx_file = f"{data_config['city']}_cell{int(data_config['cell_size'])}_feats_mx.pkl"
file_path = os.path.join(ROOT_DIR, "models/road_embs/", feats_mx_file)
with open(file_path, 'rb') as fh:
    feats_mx = pickle.load(fh)



In [ ]:
_, edge_index = cs.all_neighbour_cell_pairs_permutated_optmized()

# To torch tensor
feats_mx_torch = torch.tensor(feats_mx, dtype = torch.long, device = device)
edge_index_torch = torch.tensor(edge_index, dtype = torch.long, device = device).T

# Create PYG Data Set
from torch_geometric.data import Data
data = Data(x=feats_mx_torch, edge_index=edge_index_torch)

data

In [ ]:
# Create a fake weighted adj mx
import networkx as nx
G = nx.Graph(edge_index)
adj = nx.adjacency_matrix(G)
adj = adj.toarray()
row_sums = adj.sum(axis=1)
adj = adj / row_sums[:, np.newaxis]


In [ ]:
model = SFCModel(data, adj=adj, device=device, layers=2, add_edge_degree=False)

In [ ]:
model.train()

In [ ]:
node2vec = Node2VecModel(
    data,
    device=device,
    emb_dim=128,
    walk_length=50,
    walks_per_node=10,
    context_size=10,
)

In [ ]:
node2vec.train(20)

In [ ]:
node2vec.load_model

In [ ]:
model_emb = node2vec.load_emb()

In [ ]:
print(model_emb.shape)

# Save it for trajectory embeddings
save_path ="trajemb/models/road_embs"

# Safe embeddings
#model_emb = torch.from_numpy(model_emb)
model_name = "node2vec"
city = "porto"
add = f""
embs_file = f"{city}_cell_embs_{model_name}{add}.pkl"
embs_file = os.path.join(save_path, embs_file)
with open(embs_file, 'wb') as fh:
    pickle.dump(model_emb, fh, protocol = pickle.HIGHEST_PROTOCOL)
    print("Saved to: ", embs_file)

In [ ]:
emb_dim = 128

In [ ]:
data.x.shape

In [ ]:
# Try normalizing, without transform, ...

In [ ]:
import torch_geometric.transforms as T
transform = T.Compose([
    T.OneHotDegree(9), # training without features
    #T.ToDevice(device),
])
data = transform(data)
print(data.x.shape)
model = GAEModel(data, device=device, encoder=GATEncoder, emb_dim=128)


In [ ]:
model.train(epochs=10000)

In [ ]:
model_emb = model.load_emb()

In [ ]:
# Save it for trajectory embeddings
save_path ="trajemb/models/road_embs"

# Safe embeddings
#model_emb = torch.from_numpy(model_emb)
model_name = "gae"
city = "porto"
add = f""
embs_file = f"{city}_cell_embs_{model_name}{add}.pkl"
embs_file = os.path.join(save_path, embs_file)
# pickle
import pickle
with open(embs_file, 'wb') as fh:
    pickle.dump(model_emb, fh, protocol = pickle.HIGHEST_PROTOCOL)
    print("Saved to: ", embs_file)

In [ ]:
traj2vec = SkipGramModel(
            data,
            adj=adj,
            device=device,
            emb_dim=128,
            walk_length=50,#30
            context_size=10, #5
            walks_per_node=10, #25
            num_neg=10,
        )



In [ ]:
traj2vec.train(epochs=30)

In [ ]:
model_emb = traj2vec.load_emb()

In [ ]:
model_emb.shape

In [ ]:
# Save it for trajectory embeddings
save_path ="trajemb/models/road_embs"

# Safe embeddings
model_emb = torch.from_numpy(model_emb)
model_name = "traj_skipgram"
city = "porto"
add = f""
embs_file = f"{city}_road_embs_{model_name}{add}.pkl"
embs_file = os.path.join(save_path, embs_file)
# pickle
import pickle
with open(embs_file, 'wb') as fh:
    pickle.dump(model_emb, fh, protocol = pickle.HIGHEST_PROTOCOL)
    print("Saved to: ", embs_file)